<a href="https://colab.research.google.com/github/Abhishek2104200/QML/blob/main/Base_QSVM_QKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load your full feature dataset
file_path = '/content/drive/MyDrive/balanced_sars_cov2_variants.csv'
df = pd.read_csv(file_path)

print("Dataset shape:", df.shape)
df.head()

Dataset shape: (75000, 30)


/tmp/ipython-input-2-829872341.py:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Accession,Organism_Name,GenBank_RefSeq,Submitters,Organization,Org_location,Release_Date,Pangolin,PangoVersions,Surveillance_Sampling,...,Country,USA,Host,Tissue_Specimen_Source,Collection_Date,BioSample,BioProject,GenBank_Title,Variant_Group,WHO_Variant
0,OX474929.1,Severe acute respiratory syndrome coronavirus 2,GenBank,NaN,"Northumbria University, Newcastle upon Tyne, N...",NaN,2023-04-29,B,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,United Kingdom,NaN,Homo sapiens,NaN,2020-03-17,SAMEA7079119,PRJEB37886,Severe acute respiratory syndrome coronavirus ...,B,Alpha
1,MZ185301.1,Severe acute respiratory syndrome coronavirus 2,GenBank,"Howard,D., Batra,D., Cook,P.W., Moser,K., Pask...","Centers for Disease Control and Prevention, Re...",USA,2021-05-13,B.1.526,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,True,...,USA,ME,Homo sapiens,oronasopharynx,2021-04-29,SAMN22689283,PRJNA731148,Severe acute respiratory syndrome coronavirus ...,B,Alpha
2,OU389953.1,Severe acute respiratory syndrome coronavirus 2,GenBank,NaN,London,United Kingdom,2021-07-13,B.1.1.7,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,United Kingdom,NaN,Homo sapiens,NaN,2021-01-11,SAMEA8368730,PRJEB37886,Severe acute respiratory syndrome coronavirus ...,B,Alpha
3,OX597302.1,Severe acute respiratory syndrome coronavirus 2,GenBank,NaN,"Public Health Wales Microbiology Cardiff, Univ...",NaN,2023-05-13,B.1.177.15,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,United Kingdom,NaN,Homo sapiens,NaN,2020-12-01,SAMEA7813988,PRJEB37886,Severe acute respiratory syndrome coronavirus ...,B,Alpha
4,PP603966.1,Severe acute respiratory syndrome coronavirus 2,GenBank,"Amin,H., Wang,J., Su,M., Akther,S., Bologna,J....","NYC DOHMH, Bureau of the Public Health Laboratory",USA,2024-04-09,B.1.1.434,4.3.1/1.31/v0.1.12/0.3.19/0.6.2,NaN,...,USA,NaN,Homo sapiens,NaN,2021-02-10,SAMN20961554,PRJNA704697,Severe acute respiratory syndrome coronavirus ...,B,Alpha


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Drop purely ID-related or non-informative columns
drop_cols = [
    'Accession', 'GenBank_RefSeq', 'Submitters', 'Organization',
    'BioSample', 'BioProject', 'GenBank_Title', 'Isolate'
]
df.drop(columns=drop_cols, inplace=True, errors='ignore')

# Handle date columns: convert to datetime and then to numeric
for col in ['Release_Date', 'Collection_Date']:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    df[col] = (df[col] - df[col].min()).dt.days  # Days since earliest date

# Fill missing numerical values with median
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Fill missing categorical values with 'Unknown'
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna("Unknown")

# Encode categorical features using LabelEncoder (can switch to OneHot for tree models)
encoders = {}
# Convert boolean columns to string (so they can be handled like categories)
for col in cat_cols:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(str)
 # save if you need to inverse-transform later

# Final feature and label separation
X = df.drop(columns=['WHO_Variant'])
y = df['WHO_Variant']
y = LabelEncoder().fit_transform(y)

num_classes = len(set(y))
print("Number of unique classes in y:", num_classes)


Number of unique classes in y: 5


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Identify object-type columns (including those with 'Unknown')
object_cols = X.select_dtypes(include=['object']).columns

# Encode each using LabelEncoder
for col in object_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

In [ ]:
# Re-split with cleaned X
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
!pip install qiskit==0.43.1 qiskit-machine-learning==0.6.1

In [ ]:
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import ZZFeatureMap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import PauliFeatureMap
import numpy as np

# Set reproducibility
algorithm_globals.random_seed = 42

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Step 2: Reduce dimensionality for quantum kernel (2-4 features recommended)
pca = PCA(n_components=2)  # You can adjust to 2, 3, or 4 based on performance

X_train_reduced = pca.fit_transform(X_train_scaled)
X_test_reduced = pca.transform(X_test_scaled)

# Step 3: Define the quantum feature map and kernel
feature_map = PauliFeatureMap(feature_dimension=X_train_reduced.shape[1], reps=1, paulis=['X', 'Y', 'Z'])
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)

quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)

# Step 4: QSVM model training
qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(X_train_reduced, y_train)

# Step 5: Prediction and evaluation
y_pred = qsvc.predict(X_test_reduced)
print(classification_report(y_test, y_pred))


(60000, 2)


/tmp/ipython-input-8-3560355462.py:34: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)


In [ ]:
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import PauliFeatureMap
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from qiskit import Aer
from qiskit.utils import QuantumInstance
import numpy as np

# Set reproducibility
algorithm_globals.random_seed = 42

# Step 1: Scale features
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 2: Reduce dimensionality for quantum kernel (2-4 features recommended)
pca = PCA(n_components=2)
X_train_reduced = pca.fit_transform(X_train_scaled)
X_test_reduced = pca.transform(X_test_scaled)

# ✅ Reduce dataset size
X_train_reduced = X_train_reduced[:10000]
y_train_small = y_train[:10000]
X_test_reduced = X_test_reduced[:3000]
y_test_small = y_test[:3000]

# Step 3: Define the quantum feature map and kernel
feature_map = PauliFeatureMap(feature_dimension=2, reps=1, paulis=['X', 'Y', 'Z'])
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)

quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)

# Step 4: QSVM model training
qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(X_train_reduced, y_train_small)

# Step 5: Prediction and evaluation
y_pred = qsvc.predict(X_test_reduced)
print(classification_report(y_test_small, y_pred))

/tmp/ipython-input-10-3888566093.py:34: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)


              precision    recall  f1-score   support

           0       0.84      0.61      0.71       557
           1       0.76      0.79      0.78       631
           2       0.48      0.88      0.62       605
           3       0.69      0.45      0.54       607
           4       0.70      0.50      0.58       600

    accuracy                           0.65      3000
   macro avg       0.69      0.65      0.65      3000
weighted avg       0.69      0.65      0.65      3000



In [ ]:
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import PauliFeatureMap
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from qiskit import Aer
from qiskit.utils import QuantumInstance

import numpy as np

# Set reproducibility
algorithm_globals.random_seed = 42

# Scale data to [0, π] range
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the quantum feature map and kernel
feature_map = PauliFeatureMap(feature_dimension=X_train_scaled.shape[1], reps=1, paulis=['X', 'Y', 'Z'])
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)

quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)

# QSVM model training
qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(X_train_scaled, y_train)

# Prediction and evaluation
y_pred = qsvc.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

/tmp/ipython-input-7-1059690682.py:23: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)


In [ ]:
np.bincount(y_train)

array([40, 40, 40])

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')
clf.fit(X_train_reduced, y_train)
y_pred = clf.predict(X_test_reduced)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.82      0.90      0.86        10
           2       0.89      0.80      0.84        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30



**QKNN**

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from collections import Counter

from qiskit import Aer
from qiskit.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel

# Fix randomness
algorithm_globals.random_seed = 42
np.random.seed(42)

# 1. Scale input features to [0, π]
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Reduce to 2D for simulation
pca = PCA(n_components=2)
X_train_reduced = pca.fit_transform(X_train_scaled)
X_test_reduced = pca.transform(X_test_scaled)

# 3. Build a simple feature map
from qiskit.circuit.library import PauliFeatureMap
feature_map = PauliFeatureMap(feature_dimension=2, reps=1, paulis=['X', 'Y', 'Z'])

# feature_map = ZZFeatureMap(feature_dimension=2, reps=1)
sampler = Sampler()

# 4. Create kernel which uses default ComputeUncompute internally
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

# 5. Evaluate fidelity matrix and convert to distances
fidelity_matrix = quantum_kernel.evaluate(X_test_reduced, X_train_reduced)
distance_matrix = 1 - fidelity_matrix

# 6. Classic k-NN wrapper using quantum distance
def quantum_knn(distance_matrix, y_train, k=3):
    preds = []
    for row in distance_matrix:
        idx = np.argsort(row)[:k]
        preds.append(Counter(y_train[idx]).most_common(1)[0][0])
    return np.array(preds)

y_pred = quantum_knn(distance_matrix, y_train, k=3)

# 7. Evaluate
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.83      1.00      0.91        10
           2       1.00      0.80      0.89        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30

